# Import des librairies

In [1]:
import os
import io
import time
import datetime
import pandas as pd
import xlsxwriter
import numpy as np

# Init divers

# Init du folder emplacement des fichiers CSV

In [2]:
filepath_data = r"C:\Users\gille\JupyterProjects\Foncier\data\external"+chr(92)
filepath_processed = r"C:\Users\gille\JupyterProjects\Foncier\data\processed"+chr(92)
filepath_interim = r"C:\Users\gille\JupyterProjects\Foncier\data\interim"+chr(92)
file_xls_analyse = "Analyse_dataframe_foncier.xlsx"
list_df=list()

# Fonction Debug

In [3]:
def debug(message=" ") :
    ts = time.time()
    ts = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    print ("{} Debug -> ".format(ts)+str(message))

# Fonction de chargement et d'agrégation des fichiers txt

In [4]:
def df_get (filepath) :
    
    files = [f for f in os.listdir(filepath_data) if f.endswith(".txt")]
    
    for i, file in enumerate(files):
        debug("Agregation dans le dataframe FONCIER du fichier :" + file)
        list_df.append(pd.read_csv(filepath_data + chr(92) + file, delimiter='|', low_memory=False,
                                   decimal=",",
                                   dtype={"Surface_Carrez_du_1er_lot": np.float64,
                                         "Surface_Carrez_du_2eme_lot": np.float64,
                                         "Surface_Carrez_du_3eme_lot": np.float64,
                                         "Surface_Carrez_du_4eme_lot": np.float64,
                                         "Surface_Carrez_du_5eme_lot": np.float64}
                                  ))

        debug("Nombre de ligne chargées :" + str(len(list_df[i])))
        debug()

    debug("Fusion des Dataframes")
    df = pd.concat(list_df)
    debug()
    
    col_list = [col for col in df.columns if df[col].dtypes == "object"]
    debug("Mise en majuscule des Objects : "+str(col_list))
    for col in col_list :
        df[col]=df[col].str.upper()

    debug("Netoyage des headers de colonne")
    df.rename(columns=lambda x: x.replace(' ', '_')
              .replace('/', '_').replace('é', 'e').replace('è', 'e')
              .replace('û', 'u').replace('à', 'a').upper()
              , inplace=True)
    debug()
    return df

# Fonction d'addition des colonnes et suppression des colonnes additionées

In [5]:
def df_add_col (df,col_to_sum,col_tot) :
    # Certaine colonnes, dont la suppression est demandée maintenant, ont potentiellement déjà été supprimées parce que vide
    # Oblige à faire un examen de chaque colonne , 1 par une , et non un drop de la liste complète.
    debug ("Addition des colonnes {c} dans {t}".format(c=str(col_to_sum),t=col_tot))
    df[col_tot]=df[col_to_sum].sum(axis=1)
    debug ("suppresion des colonnes "+str(col_to_sum).upper())
    df.drop(col_to_sum,axis=1,inplace=True)
    return df
    

# Fonction de suppression des colonnes vides

In [6]:
def df_del_empty_col (df) :
    to_remove = [col for col in df.columns if df[col].value_counts().empty]
    debug("Supression des colonnes vides :"+str(to_remove))
    df.drop (to_remove,axis=1,inplace=True)
    return df

# Fonction de génération d'un fichier sample

In [7]:
def df_gen_sample (df,nb_rows,filename,filepath) :
    
    debug ("Generation du fichier d'échantillon pour analyse sous excel :")
    debug (filepath+filename)
    # Init de l'engine
    ts = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H_%M_%S')
    writer_sample = pd.ExcelWriter(filepath+str(ts)+"_"+filename+".xlsx", engine='xlsxwriter')

    #génération
    df.sample(n=5000000,random_state=1).to_csv (filepath+str(ts)+"_"+filename+'.csv')
    df.sample(n=nb_rows,random_state=1).to_excel(writer_sample,sheet_name="Sample")

    #instanciation de l'onglet "sample"
    worksheet_sample = writer_sample.sheets['Sample']

    #Auto filter des colones
    worksheet_sample.autofilter("A1:AS"+str(nb_rows-1))

    writer_sample.save()
    writer_sample.close()

# Generation du fichier d'analyse

In [8]:
def df_gen_analyse (df,filepath,filename) :
    debug("Generation du fichier d'analyse")
    path = filepath + filename
    writer_analyse = pd.ExcelWriter(path, engine='xlsxwriter')
    debug("Creation de l'onglet INFO ")
    workbook_foncier  = writer_analyse.book
    worksheet_info=workbook_foncier.add_worksheet("INFO")
    worksheet_drop=workbook_foncier.add_worksheet("DROP")
    worksheet_describep=workbook_foncier.add_worksheet("DESCRIBE")
    
    debug("Alimentation de l'onglet INFO ")
    cell_format = workbook_foncier.add_format()
    cell_format.set_align('vjustify')
    buf = io.StringIO()
    df.info(buf=buf)
    s = buf.getvalue()
    worksheet_info.write(0,0,s,cell_format)
    
    debug("Generation des counts par colonne")
    for i,col in enumerate(df.columns):
        debug ("Ajout de la colonne+count au fichier Excel : "+col)
        df[col].value_counts().to_excel(writer_analyse,sheet_name=col)
            
    writer_analyse.save()
    writer_analyse.close()
    debug("Sauvegarde de l'analyse sous excel")


# Main

In [9]:
debug ("Start")
debug ("************** Chargement, agrégation, et suppression des colonnes vides")
debug ()
df_foncier = df_del_empty_col(df_get (filepath_data))
debug ("End")

2021-05-13 22:20:17 Debug -> Start
2021-05-13 22:20:17 Debug -> ************** Chargement, agrégation, et suppression des colonnes vides
2021-05-13 22:20:17 Debug ->  
2021-05-13 22:20:17 Debug -> Agregation dans le dataframe FONCIER du fichier :valeursfoncieres-2016_short.txt
2021-05-13 22:20:17 Debug -> Nombre de ligne chargées :20
2021-05-13 22:20:17 Debug ->  
2021-05-13 22:20:17 Debug -> Agregation dans le dataframe FONCIER du fichier :valeursfoncieres-2017_short.txt
2021-05-13 22:20:17 Debug -> Nombre de ligne chargées :20
2021-05-13 22:20:17 Debug ->  
2021-05-13 22:20:17 Debug -> Fusion des Dataframes
2021-05-13 22:20:17 Debug ->  
2021-05-13 22:20:17 Debug -> Mise en majuscule des Objects : ['Date mutation', 'Nature mutation', 'Type de voie', 'Code voie', 'Voie', 'Commune', 'Section', 'Type local', 'Nature culture']
2021-05-13 22:20:17 Debug -> Netoyage des headers de colonne
2021-05-13 22:20:17 Debug ->  
2021-05-13 22:20:17 Debug -> Supression des colonnes vides :['CODE_SERV

In [10]:
debug ("************** Addition des surfaces Carrez")

df_foncier=df_add_col (df_foncier,["SURFACE_CARREZ_DU_1ER_LOT",
                                    "SURFACE_CARREZ_DU_2EME_LOT",
                                    "SURFACE_CARREZ_DU_3EME_LOT",
                                    "SURFACE_CARREZ_DU_4EME_LOT",
                                    "SURFACE_CARREZ_DU_5EME_LOT"],"TOTAL_SURF_CARREZ")

debug("end")

2021-05-13 22:20:17 Debug -> ************** Addition des surfaces Carrez
2021-05-13 22:20:17 Debug -> Addition des colonnes ['SURFACE_CARREZ_DU_1ER_LOT', 'SURFACE_CARREZ_DU_2EME_LOT', 'SURFACE_CARREZ_DU_3EME_LOT', 'SURFACE_CARREZ_DU_4EME_LOT', 'SURFACE_CARREZ_DU_5EME_LOT'] dans TOTAL_SURF_CARREZ


KeyError: "['SURFACE_CARREZ_DU_4EME_LOT', 'SURFACE_CARREZ_DU_5EME_LOT', 'SURFACE_CARREZ_DU_3EME_LOT'] not in index"

debug ("************** Suppression des colonnes inutiles")

to_remove =['No_disposition','No_voie','B_T_Q','Type_de_voie','Voie','Code_postal','Commune','Code_departement','Prefixe_de_section',
             'Section','No_plan','No_Volume','1er_lot','2eme_lot','3eme_lot','4eme_lot','5eme_lot','Code_type_local']

for col in to_remove :
    try :
        df_foncier.drop(col,axis=1,inplace=True)
    except :
        debug ("Colonne {c} non présente dans le data frame".format(c=col))

debug ("************** Génération du fichier de Sample")
df_gen_sample (df_foncier,500000,"Sample Foncier",filepath_processed)
debug ("end")
debug ()

# Sauvegarde du DF au format CSV

df_foncier.to_csv (filepath_interim+'df_foncier.txt')

In [ ]:
df_foncier.head()